# Decorators

A Python decorator is a specific change to the Python syntax that allows us to more conveniently alter functions and methods.

In [1]:
def decorator(f):
    print "Decorator called with argument:", f
    
    def wrapper():
        print "Wrapper called"
        f()

    return wrapper

@decorator
def nothing_doer():
    print "Decorated function called"
    
nothing_doer()
print(nothing_doer.__name__)

Decorator called with argument: <function nothing_doer at 0x10ec4ede8>
Wrapper called
Decorated function called
wrapper


 It's syntax sugar, equivalent code:

In [2]:
def decorator(f):
    print "Decorator called with argument:", f

    def wrapper():
        print "Wrapper called"
        f()
        
    return wrapper

def nothing_doer():
    print "Decorated function called"
    
wrapped = decorator(nothing_doer)
wrapped()

Decorator called with argument: <function nothing_doer at 0x10ec4ec08>
Wrapper called
Decorated function called


Nested decorators:

In [3]:
def decorator1(f):
    print "Decorator 1 called"
    
    def wrapper():
        print "Decorator 1 wrapper called"
        f()
        
    return wrapper
        
def decorator2(f):
    print "Decorator 2 called"
    
    def wrapper():
        print "Decorator 2 wrapper called"
        f()
        
    return wrapper

@decorator1
@decorator2
def nothing_doer():
    print "Decorated function called"
    
nothing_doer()

Decorator 2 called
Decorator 1 called
Decorator 1 wrapper called
Decorator 2 wrapper called
Decorated function called


Parametrized decorator:

In [4]:
myvar = "World"

def superdecorator(message, subject):
    def decorator(f):
        def wrapper():
            print "Wrapper called: " + message + " " + subject
            f()

        return wrapper
    
    return decorator

@superdecorator("Hello", myvar)
def nothing_doer():
    print "Decorated function called"
    
nothing_doer()

Wrapper called: Hello World
Decorated function called


Class property decorator:

In [5]:
class Celsius:
    def __init__(self, temperature = 0):
        self._temperature = temperature

    def to_fahrenheit(self):
        return (self.temperature * 1.8) + 32

    @property
    def temperature(self):
        print "Getting value"
        return self._temperature

    @temperature.setter
    def temperature(self, value):
        if value < -273:
            raise ValueError("Temperature below -273 is not possible")
        print "Setting value"
        self._temperature = value
        
c = Celsius()
c.temperature = 36.6
c.temperature

36.6

How it works:

[Documentation on property()](https://docs.python.org/3/library/functions.html#property)

First, equivalent code:

In [6]:
class MyClass:
    def getter(self):
        print "Getter called"

    def setter(self, arg):
        print "Setter called: {}".format(arg)

    x = property(getter, setter)

m = MyClass()
m.x
m.x = 12

Getter called


Let's write our own property():

In [7]:
class MyProperty:
    def __init__(self, f):
        self._getter = f
        
    def __get__(self, instance, klass):
        return self._getter(instance)
       
    def __set__(self, instance, value):
        return self._setter(instance, value)
        
    def setter(self, f):
        self._setter = f

def myproperty(f):
    return MyProperty(f)

class MyClass:
    def __init__(self, x):
        self._x = x
    
    @myproperty
    def x(self):
        print "Getter called: {}".format(self._x)
        return self._x

    @x.setter
    def setter(self, arg):
        print "Setter called: {}".format(arg)
        self._x = arg

m = MyClass(10)
print(m.x)

m.x = 12
print(m.x)

<__main__.MyProperty instance at 0x10ec9ccb0>
12


## Memoize pattern

Imagine factorial() contains some expensive computations:

In [8]:
cache = {}

def memoize(f):
    def wrapper(x):
        if x in cache:
            r = cache[x]
            print "Cache hit: {}, result is {}".format(x, r)
            return r
        
        r = f(x)
        cache[x] = r
        
        return r
        
    return wrapper

@memoize
def some_expensive_computations(x):
    r = x ** 2
    print "Original function called with argument: {}, result is {}".format(x, r)
    return r

print(some_expensive_computations(3))
print(some_expensive_computations(3))
print(some_expensive_computations(3))

print(some_expensive_computations(4))

Original function called with argument: 3, result is 9
9
Cache hit: 3, result is 9
9
Cache hit: 3, result is 9
9
Original function called with argument: 4, result is 16
16


## Module 'functools'

[Docs](https://docs.python.org/3/library/functools.html)

` @functools.lru_cache()`: Don't reinvent the wheel + some goodies

In [9]:
import functools

@functools.lru_cache(maxsize=6)
def factorial(x):
    return 1 if x == 1 else x * factorial(x - 1)

for n in range(1, 6):
    factorial(n)

factorial.cache_info()

AttributeError: 'module' object has no attribute 'lru_cache'

**functools.partial()**: [Stackoverflow: What is currying](https://stackoverflow.com/questions/36314/what-is-currying)

In [ ]:
def manyargs(a, b, c):
    print "a={}, b={}, c={}".format(a, b, c)
    
p1 = functools.partial(manyargs, 1)
p2 = functools.partial(p1, 2)

p2(3)

In [ ]:
p1 = functools.partial(manyargs, c=1)
p2 = functools.partial(p1, b=2)

p2(a=3)

` @functools.wraps()`: fixes wrapped function metadata

In [ ]:
def decorator(f):
    @functools.wraps(f)
    def wrapper():
        print "Wrapper called"
        f()
        
    return wrapper
    
@decorator
def nothing_doer():
    print "Decorated function called"
    
nothing_doer()

print(nothing_doer.__name__)